In [6]:
import pandas as pd
import re
import numpy as np

In [2]:
combined_camp_keyword_match_df = pd.read_csv("combined_camp_keyword&variations_df_163519.csv")
combined_camp_keyword_match_df.head()

,keyword_id,marketing_campaign_id,text,match_type,bidding_value_usd,keyword_variation,request,response,RR
0,135196,375534,sheet mask,PHRASE,0.4059,sheet mask,14900.0,14854.0,99.691275
1,135196,375534,sheet mask,PHRASE,0.4059,garnier sheet mask,1069.0,406.0,37.979420
2,135196,375534,sheet mask,PHRASE,0.4059,good vibes sheet mask,657.0,655.0,99.695586
3,135196,375534,sheet mask,PHRASE,0.4059,sheet mask combo,624.0,623.0,99.839744
4,135196,375534,sheet mask,PHRASE,0.4059,lakme face sheet mask,571.0,214.0,37.478109


In [7]:
# First cosider top 5 categories based on advertisable sku count as top categories
key_cat_df= pd.read_csv("keyword_category_data_v2_163519.csv")
#Concatination
# key_cat_df["cat_concat"] = key_cat_df[['category_l1', 'category_l2', 'category_l3', 'category_l4', 'category_l5', 'category_l6',
#        'category_l7', 'category_l8']].apply(lambda row: '>'.join([str(x).strip().lower() for x in row]), axis=1)
key_cat_df["cat_concat"] = key_cat_df[['category_l1', 'category_l2', 'category_l3', 'category_l4', 'category_l5', 'category_l6',
                                       'category_l7', 'category_l8']].apply(lambda row: '>'.join([str(x).strip().lower() for x in row if pd.notnull(x) and x != 'nan']), axis=1)

key_cat_df.head()

,keyword,category_l1,category_l2,category_l3,category_l4,category_l5,category_l6,category_l7,category_l8,source,count,tag,advertisable_sku_count,cat_concat
0,lipstick,makeup,lip makeup,lipstick,NaN,NaN,NaN,NaN,NaN,auto,574495.20,advertisable_skus_present,4570008.0,makeup>lip makeup>lipstick
1,sunscreen,skincare,sunscreen,sunscreen,NaN,NaN,NaN,NaN,NaN,auto,175270.80,advertisable_skus_present,155862.0,skincare>sunscreen>sunscreen
2,marslipstick,makeup,lip makeup,lipstick,NaN,NaN,NaN,NaN,NaN,auto,171910.00,advertisable_skus_present,2419416.0,makeup>lip makeup>lipstick
3,lipstick,makeup,lip makeup,lip liner,NaN,NaN,NaN,NaN,NaN,manual,164098.44,advertisable_skus_present,99.0,makeup>lip makeup>lip liner
4,lipstick,makeup,lip makeup,lip gloss,NaN,NaN,NaN,NaN,NaN,manual,160261.06,advertisable_skus_present,319.0,makeup>lip makeup>lip gloss


In [9]:
key_cat_df.shape

(807150, 14)

In [10]:
# Sort the DataFrame by 'keyword' and 'advertisable_sku_cou' in descending order
sorted_df = key_cat_df.sort_values(by=['keyword', 'count'], ascending=[True, False])
# Group by 'keyword' and apply head(5) to get the top 5 rows per group
top_categories_df = sorted_df.groupby('keyword').head(5).reset_index(drop=True)
top_categories_df.shape

(597631, 14)

In [11]:
top_categories_df = top_categories_df[['keyword', 'cat_concat']]
top_categories_df.head()

,keyword,cat_concat
0,-bhilaiboardtohamen,skincare>toner & face mist>toner
1,-blouse,makeup>lip makeup>lipstick
2,-blouse,fragrance>gift and value set - fragrance>fragr...
3,-brandedlipstick,makeup>lip makeup>lipstick
4,-colorkalambasticklips,makeup>lip makeup>lipstick


In [12]:
# Now, group by 'keyword' and aggregate the 'category' into a list
top_categories_df = top_categories_df.groupby('keyword')['cat_concat'].apply(list).reset_index()

# Print the resulting DataFrame
top_categories_df.head()

,keyword,cat_concat
0,-bhilaiboardtohamen,[skincare>toner & face mist>toner]
1,-blouse,"[makeup>lip makeup>lipstick, fragrance>gift an..."
2,-brandedlipstick,[makeup>lip makeup>lipstick]
3,-colorkalambasticklips,[makeup>lip makeup>lipstick]
4,-copykarneka,[skincare>mask>face pack & face mask]


In [13]:
top_categories_df[top_categories_df['keyword']=='bathingpowder']

,keyword,cat_concat
24646,bathingpowder,"[skincare>mask>face pack & face mask, skincare..."


In [14]:
camp_cat_df = pd.read_csv("purplle_campaign_SKUs_category_11thAug24.tsv000", sep='\t')
camp_cat_df.shape

(2246, 12)

In [15]:
# Now, group by 'keyword' and aggregate the 'category' into a list
camp_cat_df["concat"] = camp_cat_df[['category_l1', 'category_l2', 'category_l3', 'category_l4', 'category_l5', 'category_l6',
       'category_l7', 'category_l8']].apply(lambda row: '>'.join([str(x).strip().lower() for x in row if pd.notnull(x) and x != 'nan']), axis=1)
# .apply(lambda row: '>'.join([str(x).strip().lower() for x in row]), axis=1)

camp_cat_df = camp_cat_df.groupby(['merchant_id', 'marketing_campaign_id', 'campaign_id'])['concat'].apply(list).reset_index()

# Print the resulting DataFrame
camp_cat_df.head()

,merchant_id,marketing_campaign_id,campaign_id,concat
0,4,664297,508919,[makeup>lip makeup>lipstick]
1,4,664302,508922,[makeup>face makeup>foundation]
2,4,664316,508938,[makeup>nail>nail polish]
3,4,664331,508943,[makeup>lip makeup>lipstick]
4,4,671007,513445,[makeup>lip makeup>lipstick]


In [16]:
camp_cat_df.shape

(1413, 4)

In [17]:
# combined_camp_keyword_match_df = combined_camp_keyword_match_df.drop(['marketplace_client_id', 'agency_id', 'is_negative', 'status_type', 'bidding_value', 'bidding_currency'], axis=1)

In [18]:
camp_key_cat_df = pd.merge(combined_camp_keyword_match_df, camp_cat_df[['merchant_id','marketing_campaign_id', 'concat']], on='marketing_campaign_id', how='left')
camp_key_cat_df.shape

(9979768, 11)

In [19]:
# import re

camp_key_cat_df['trimmed_variation'] = camp_key_cat_df['keyword_variation'].apply(lambda x: re.sub(r'\s+', '', x.strip()) if isinstance(x, str) else x)


In [20]:
camp_key_cat_df.head()

,keyword_id,marketing_campaign_id,text,match_type,bidding_value_usd,keyword_variation,request,response,RR,merchant_id,concat,trimmed_variation
0,135196,375534,sheet mask,PHRASE,0.4059,sheet mask,14900.0,14854.0,99.691275,NaN,NaN,sheetmask
1,135196,375534,sheet mask,PHRASE,0.4059,garnier sheet mask,1069.0,406.0,37.979420,NaN,NaN,garniersheetmask
2,135196,375534,sheet mask,PHRASE,0.4059,good vibes sheet mask,657.0,655.0,99.695586,NaN,NaN,goodvibessheetmask
3,135196,375534,sheet mask,PHRASE,0.4059,sheet mask combo,624.0,623.0,99.839744,NaN,NaN,sheetmaskcombo
4,135196,375534,sheet mask,PHRASE,0.4059,lakme face sheet mask,571.0,214.0,37.478109,NaN,NaN,lakmefacesheetmask


In [21]:
camp_key_cat_df[camp_key_cat_df['text']=='lipstick']

,keyword_id,marketing_campaign_id,text,match_type,bidding_value_usd,keyword_variation,request,response,RR,merchant_id,concat,trimmed_variation
215799,434341,395599,lipstick,EXACT,0.0947,lipstick,213661.0,212057.0,99.249278,22.0,[makeup>lip makeup>lipstick],lipstick
215804,434346,395589,lipstick,EXACT,0.0812,lipstick,213661.0,212057.0,99.249278,22.0,[makeup>lip makeup>lipstick],lipstick
404740,472445,612161,lipstick,BROAD,0.2706,cosrx lipstick,1.0,0.0,0.000000,1299.0,[personal care>shaving & hair removal>face & e...,cosrxlipstick
404741,472445,612161,lipstick,BROAD,0.2706,hair colour lipstick types,2.0,0.0,0.000000,1299.0,[personal care>shaving & hair removal>face & e...,haircolourlipsticktypes
404742,472445,612161,lipstick,BROAD,0.2706,hair colour lipstick hair colour,2.0,0.0,0.000000,1299.0,[personal care>shaving & hair removal>face & e...,haircolourlipstickhaircolour
...,...,...,...,...,...,...,...,...,...,...,...,...
9362488,1571671,854726,lipstick,PHRASE,0.2341,electric pink lipstick,1.0,1.0,100.000000,779.0,[makeup>lip makeup>lipstick],electricpinklipstick
9362489,1571671,854726,lipstick,PHRASE,0.2341,elee18 lipstick,1.0,1.0,100.000000,779.0,[makeup>lip makeup>lipstick],elee18lipstick
9362490,1571671,854726,lipstick,PHRASE,0.2341,ele jude lipstick,1.0,0.0,0.000000,779.0,[makeup>lip makeup>lipstick],elejudelipstick
9362491,1571671,854726,lipstick,PHRASE,0.2341,- - color ke lipstick shade,1.0,0.0,0.000000,779.0,[makeup>lip makeup>lipstick],--colorkelipstickshade


In [22]:
# Join top categories data
camp_key_cat_top_cat_df = pd.merge(camp_key_cat_df, top_categories_df, left_on='trimmed_variation', right_on='keyword', how='left')
camp_key_cat_top_cat_df.head()

,keyword_id,marketing_campaign_id,text,match_type,bidding_value_usd,keyword_variation,request,response,RR,merchant_id,concat,trimmed_variation,keyword,cat_concat
0,135196,375534,sheet mask,PHRASE,0.4059,sheet mask,14900.0,14854.0,99.691275,NaN,NaN,sheetmask,sheetmask,"[skincare>mask>sheet mask, skincare>mask>peel ..."
1,135196,375534,sheet mask,PHRASE,0.4059,garnier sheet mask,1069.0,406.0,37.979420,NaN,NaN,garniersheetmask,garniersheetmask,"[skincare>mask>sheet mask, skincare>mask>peel ..."
2,135196,375534,sheet mask,PHRASE,0.4059,good vibes sheet mask,657.0,655.0,99.695586,NaN,NaN,goodvibessheetmask,goodvibessheetmask,"[skincare>mask>face pack & face mask, skincare..."
3,135196,375534,sheet mask,PHRASE,0.4059,sheet mask combo,624.0,623.0,99.839744,NaN,NaN,sheetmaskcombo,sheetmaskcombo,"[skincare>mask>sheet mask, skincare>mask>peel ..."
4,135196,375534,sheet mask,PHRASE,0.4059,lakme face sheet mask,571.0,214.0,37.478109,NaN,NaN,lakmefacesheetmask,lakmefacesheetmask,"[skincare>mask>sheet mask, skincare>mask>peel ..."


In [23]:
camp_key_cat_top_cat_df = camp_key_cat_top_cat_df.rename(columns={'concat':'camp_category', 'cat_concat':'top_category'})

In [24]:
camp_key_cat_top_cat_df[camp_key_cat_top_cat_df['text']=='lipstick']

,keyword_id,marketing_campaign_id,text,match_type,bidding_value_usd,keyword_variation,request,response,RR,merchant_id,camp_category,trimmed_variation,keyword,top_category
215799,434341,395599,lipstick,EXACT,0.0947,lipstick,213661.0,212057.0,99.249278,22.0,[makeup>lip makeup>lipstick],lipstick,lipstick,"[makeup>lip makeup>lipstick, makeup>lip makeup..."
215804,434346,395589,lipstick,EXACT,0.0812,lipstick,213661.0,212057.0,99.249278,22.0,[makeup>lip makeup>lipstick],lipstick,lipstick,"[makeup>lip makeup>lipstick, makeup>lip makeup..."
404740,472445,612161,lipstick,BROAD,0.2706,cosrx lipstick,1.0,0.0,0.000000,1299.0,[personal care>shaving & hair removal>face & e...,cosrxlipstick,NaN,NaN
404741,472445,612161,lipstick,BROAD,0.2706,hair colour lipstick types,2.0,0.0,0.000000,1299.0,[personal care>shaving & hair removal>face & e...,haircolourlipsticktypes,NaN,NaN
404742,472445,612161,lipstick,BROAD,0.2706,hair colour lipstick hair colour,2.0,0.0,0.000000,1299.0,[personal care>shaving & hair removal>face & e...,haircolourlipstickhaircolour,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9362488,1571671,854726,lipstick,PHRASE,0.2341,electric pink lipstick,1.0,1.0,100.000000,779.0,[makeup>lip makeup>lipstick],electricpinklipstick,electricpinklipstick,[makeup>lip makeup>lipstick]
9362489,1571671,854726,lipstick,PHRASE,0.2341,elee18 lipstick,1.0,1.0,100.000000,779.0,[makeup>lip makeup>lipstick],elee18lipstick,elee18lipstick,[makeup>lip makeup>lipstick]
9362490,1571671,854726,lipstick,PHRASE,0.2341,ele jude lipstick,1.0,0.0,0.000000,779.0,[makeup>lip makeup>lipstick],elejudelipstick,NaN,NaN
9362491,1571671,854726,lipstick,PHRASE,0.2341,- - color ke lipstick shade,1.0,0.0,0.000000,779.0,[makeup>lip makeup>lipstick],--colorkelipstickshade,NaN,NaN


In [37]:
camp_key_cat_top_cat_df['camp_category'].apply(lambda x: x is None or (isinstance(x, str) and x.strip() == '') or (isinstance(x, list) and len(x) == 0)).sum()

2473311

In [26]:
camp_key_cat_top_cat_df.dtypes

keyword_id                 int64
marketing_campaign_id      int64
text                      object
match_type                object
bidding_value_usd        float64
keyword_variation         object
request                  float64
response                 float64
RR                       float64
merchant_id              float64
camp_category             object
trimmed_variation         object
keyword                   object
top_category              object
dtype: object

In [55]:
# camp_key_cat_top_cat_df['camp_category'] = camp_key_cat_top_cat_df['camp_category'].replace('nan', '', regex=False)


In [52]:
# # Function to clean the category path
# def clean_category_path(path):
#     if isinstance(path, str):
#         # Remove square brackets and split by '>'
#         parts = path.strip('[]').split('>')
#         # Filter out 'nan' values and join the remaining parts
#         cleaned_path = '>'.join([part for part in parts if part.strip().lower() != 'nan'])
#         return cleaned_path
#     return path


# # Apply the function to clean the category paths
# camp_key_cat_top_cat_df['camp_category'] = camp_key_cat_top_cat_df['camp_category'].apply(clean_category_path)
# camp_key_cat_top_cat_df['top_category'] = camp_key_cat_top_cat_df['top_category'].apply(clean_category_path)


In [30]:
#  camp_key_cat_top_cat_df['camp_category'].apply(clean_category_path)
# Ensure the categories are lists and handle NaN values
camp_key_cat_top_cat_df['camp_category'] = camp_key_cat_top_cat_df['camp_category'].apply(lambda x: x if isinstance(x, list) else [])
camp_key_cat_top_cat_df['top_category'] = camp_key_cat_top_cat_df['top_category'].apply(lambda x: x if isinstance(x, list) else [])


In [31]:
# Create a new column 'category_in_top' that checks if any campaign category is in the top categories
camp_key_cat_top_cat_df['category_in_top'] = camp_key_cat_top_cat_df.apply(lambda row: any(cat in row['top_category'] for cat in row['camp_category']), axis=1)
# camp_key_cat_top_cat_df['category_in_top'].value_counts()

<bound method IndexOpsMixin.value_counts of 0          False
1          False
2          False
3          False
4          False
           ...  
9979763    False
9979764    False
9979765    False
9979766    False
9979767     True
Name: category_in_top, Length: 9979768, dtype: bool>

In [32]:
camp_key_cat_top_cat_df['category_in_top'].value_counts()

False    8688087
True     1291681
Name: category_in_top, dtype: int64

In [34]:
camp_key_cat_top_cat_df[camp_key_cat_top_cat_df['category_in_top']==False]

,keyword_id,marketing_campaign_id,text,match_type,bidding_value_usd,keyword_variation,request,response,RR,merchant_id,camp_category,trimmed_variation,keyword,top_category,category_in_top
0,135196,375534,sheet mask,PHRASE,0.4059,sheet mask,14900.0,14854.0,99.691275,NaN,[],sheetmask,sheetmask,"[skincare>mask>sheet mask, skincare>mask>peel ...",False
1,135196,375534,sheet mask,PHRASE,0.4059,garnier sheet mask,1069.0,406.0,37.979420,NaN,[],garniersheetmask,garniersheetmask,"[skincare>mask>sheet mask, skincare>mask>peel ...",False
2,135196,375534,sheet mask,PHRASE,0.4059,good vibes sheet mask,657.0,655.0,99.695586,NaN,[],goodvibessheetmask,goodvibessheetmask,"[skincare>mask>face pack & face mask, skincare...",False
3,135196,375534,sheet mask,PHRASE,0.4059,sheet mask combo,624.0,623.0,99.839744,NaN,[],sheetmaskcombo,sheetmaskcombo,"[skincare>mask>sheet mask, skincare>mask>peel ...",False
4,135196,375534,sheet mask,PHRASE,0.4059,lakme face sheet mask,571.0,214.0,37.478109,NaN,[],lakmefacesheetmask,lakmefacesheetmask,"[skincare>mask>sheet mask, skincare>mask>peel ...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9979762,1628934,867685,Mist,PHRASE,0.9592,nybae lip mist,1.0,0.0,0.000000,2319.0,[fragrance>fragrance for women>body mist & spr...,nybaelipmist,NaN,[],False
9979763,1628934,867685,Mist,PHRASE,0.9592,nykka mist,1.0,0.0,0.000000,2319.0,[fragrance>fragrance for women>body mist & spr...,nykkamist,NaN,[],False
9979764,1628934,867685,Mist,PHRASE,0.9592,nyka cherry blossom mist,1.0,0.0,0.000000,2319.0,[fragrance>fragrance for women>body mist & spr...,nykacherryblossommist,NaN,[],False
9979765,1628934,867685,Mist,PHRASE,0.9592,elf matte magic mist and set,1.0,0.0,0.000000,2319.0,[fragrance>fragrance for women>body mist & spr...,elfmattemagicmistandset,elfmattemagicmistandset,"[makeup>face makeup>setting spray, powder & fi...",False


In [38]:
camp_key_cat_top_cat_df[camp_key_cat_top_cat_df['category_in_top']==True]

,keyword_id,marketing_campaign_id,text,match_type,bidding_value_usd,keyword_variation,request,response,RR,merchant_id,camp_category,trimmed_variation,keyword,top_category,category_in_top
20016,389279,551819,Pilgrim Serum,BROAD,0.4059,pilgrim serum,3857.0,3682.0,95.462795,2368.0,"[skincare>moisturizer>serum & essence, skincar...",pilgrimserum,pilgrimserum,"[skincare>moisturizer>serum & essence, skin ca...",True
20017,389279,551819,Pilgrim Serum,BROAD,0.4059,pilgrim face serum,547.0,527.0,96.343693,2368.0,"[skincare>moisturizer>serum & essence, skincar...",pilgrimfaceserum,pilgrimfaceserum,"[skincare>moisturizer>serum & essence, skin ca...",True
20019,389279,551819,Pilgrim Serum,BROAD,0.4059,pilgrim 24k gold serum,351.0,333.0,94.871795,2368.0,"[skincare>moisturizer>serum & essence, skincar...",pilgrim24kgoldserum,pilgrim24kgoldserum,"[skincare>moisturizer>serum & essence, skin ca...",True
20022,389279,551819,Pilgrim Serum,BROAD,0.4059,pilgrim vitamin c serum,197.0,192.0,97.461929,2368.0,"[skincare>moisturizer>serum & essence, skincar...",pilgrimvitamincserum,pilgrimvitamincserum,"[skincare>moisturizer>serum & essence, skin ca...",True
20023,389279,551819,Pilgrim Serum,BROAD,0.4059,pilgrim brightening face serum,162.0,156.0,96.296296,2368.0,"[skincare>moisturizer>serum & essence, skincar...",pilgrimbrighteningfaceserum,pilgrimbrighteningfaceserum,"[skincare>moisturizer>serum & essence, skincar...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9979667,1628934,867685,Mist,PHRASE,0.9592,aqualogika mist,1.0,0.0,0.000000,2319.0,[fragrance>fragrance for women>body mist & spr...,aqualogikamist,aqualogikamist,[fragrance>fragrance for women>body mist & spr...,True
9979671,1628934,867685,Mist,PHRASE,0.9592,aqualigica mist,1.0,1.0,100.000000,2319.0,[fragrance>fragrance for women>body mist & spr...,aqualigicamist,aqualigicamist,[fragrance>fragrance for women>perfume for wom...,True
9979691,1628934,867685,Mist,PHRASE,0.9592,smokiiee vanilla mist,1.0,0.0,0.000000,2319.0,[fragrance>fragrance for women>body mist & spr...,smokiieevanillamist,smokiieevanillamist,[fragrance>fragrance for women>body mist & spr...,True
9979724,1628934,867685,Mist,PHRASE,0.9592,hair mist for smell,1.0,0.0,0.000000,2319.0,[fragrance>fragrance for women>body mist & spr...,hairmistforsmell,hairmistforsmell,[fragrance>fragrance for women>body mist & spr...,True
